# 第二次课后练习

<span style="color:black; font-weight:bold;">请将作业文件命名为 第二次课后练习+姓名+学号.ipynb, 例如 第二次课后练习+张三+1000000000.ipynb</span>

<span style="color:black; font-weight:bold;">在作业过程中觉得有心得或者自己拓展学习到有价值内容的，可以在文件名最后加一个#号。例如第二次课后练习+张三+1000000000+#.ipynb</span>

本次作业提交截止为周四晚。建议能完成的尽量提前提交以方便作业讲评。

## 零、 阅读代码预测-观察执行结果

In [ ]:
n = 2
def func():
    print(n)

func()

def func2():
    print(n)
    n+=1
func2()

预测结果为

```bash
2
UnboundLocalError
```

与实际相符。

`func2()`的主要问题是函数中存在赋值行为。当Python Interpreter发现函数中的赋值行为时，会自动将n定义为局部变量而非全局变量，此时全局变量的n被遮蔽，`print(n)`无法访问到`n`，因而报错。

In [ ]:
#函数变量的作用域
def scope_test():
    def test_local():
        spam="Local" 
    def test_nonlocal():
        nonlocal spam
        spam="Nonlocal"
    def test_global():
        global spam
        spam="Global"
    spam="Test"
    test_local()
    print("1"+spam)
    test_global()
    print("2"+spam)
    test_nonlocal()
    print("3"+spam)


try:
    print(spam)
except:
    print("not defined")
scope_test()
print("4"+spam)

预测结果为：

```bash
not defined

1Test

2Test

3Nonlocal

4Global
```

与实际相符。

在最开始全局spam没有定义，因此出现not defined。随后在scope_test中，
1. `test_local`函数只是在自己的frame定义了一个`spam`，对`scope_test`以及全局没有影响，因此输出为`1Test`。

![test_local](./photo1.png)、

2. `test_global`函数在全局的frame定义了`spam`，对`scope_test`没有影响，因此输出为`2Test`。

![test_global](./photo2.png)

3. `test_nonlocal`函数在自身没有找到`spam`的定义，于是在`scope_test`中寻找，找到并更改其值为`Nonlocal`，所以输出为`3Nonlocal`。

![test_nonlocal](./photo3.png)

4. 最后输出全局的`4Global`。

In [ ]:
def is_palme(n):
    n=str(n)
    m=n[::-1]
    return n==m

L = [12321,12343,213,242,433]

list(filter(lambda s: is_palme(s), L))

预期结果为[12321, 242]，与实际相符。

`is_palme`函数目的在于检测回环数，左右读起来都一样，m相当于把n从右到左书写一遍。

In [ ]:
def filter_data(data, threshold):
    for item in data:
        if data.index(item) > threshold:
            yield item

data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

filtered_data5 = filter_data(data, 5)

lst = [x for x in  filtered_data5]
lst

预测结果为[7, 8, 9, 10]与实际相符。

这里想要得到列表中index大于5的元素，注意列表的元素从0开始。

In [ ]:
def fla(lst):
    for item in lst:
        if isinstance(item, list):  # 如果当前元素仍然是列表，递归调用fla
            yield from fla(item)
        else:
            yield item

nes_list = [[[1, 2], [3, 4]], 5, [6, 7, [8, 9]]]

for i in fla(nes_list):
    print(i)

预期结果为

```bash
1
2
3
4
5
6
7
8
9
```

与实际相符。这里fla利用递归调用，不断从列表中提取出元素。

reduce的作用是将一个序列的值合并成一个值，如`reduce(lambda x, y: x+y, [1, 2, 3, 4, 5])`等价于`((((1+2)+3)+4)+5)`


In [ ]:
from functools import reduce

text = """You are right, yes you are right"""

words = map(lambda s: s.lower(), filter(lambda s: s.isalpha(), text.split()))

fre = reduce(lambda d, word: {**d, word: d.get(word, 0) + 1}, words, {})

print(fre)

预期结果是{'you': 2, 'are': 2, 'yes': 1, 'right': 1}与实际相符。

`words`首先将`text`按照空格分割，并去除掉非字母组成的字符串（去除掉`right,`），然后再对每个单词的大写字母转换为小写字母。

`reduce`函数首先初始化`fre`为一个空的字典，随后对于`words`中的每个单词（尽管`words`是map类型），利用原来的`d`（lambda函数中的参数，应当认为是字典），如果已经存在`word`则加一，如果不存在则创建并初始化为1，这里`**d`可以获得`d`已经拥有的所有对应key-index串。

其实本质上说这个代码块对单词出现数量进行统计，但存在小小的bug，`right,`会被认为不是单词

In [ ]:
# 实现函数组合器

def compose(*funcs):
    return reduce(lambda f,g: lambda x: f(g(x)), funcs)

add1 = lambda x: x+1
mul2 = lambda x: x*2

compose(mul2, add1)(3)

预期结果是8，与实际结果相符。

`compose`函数获得的参数数量不定（`*funcs`）。reduce函数从itertable中获取前两个并调用函数，随后每次再从后面的iterable获取一个项并作为第二个参数，前面函数计算得到的结果作为第一个参数继续调用函数，这里x=3，内层计算x+1，外面计算(x+1)*2，最后得到结果。

In [ ]:
funcs = [lambda x: x+i for i in range(3)]
print([f(10) for f in funcs])

预期结果为[12, 12, 12]与实际相符。

这里的lambda函数定义后出现late binding，只有在这个lambda函数被调用时interpreter才回去查找对应的i的值，然而此时上面的i循环已经结束，i=2，因而当`funcs`的三个lambda函数调用时，得到的都是12.

In [ ]:
def counter_factory(initial=0):
    count = initial  # 闭包变量存储状态
    def increment(step=1):
        nonlocal count
        count += step
        return count
    def reset():
        nonlocal count
        count = 0
        return count
    def get():
        return count
    return {'increment': increment, 'reset': reset, 'get': get}

counter1 = counter_factory(10)
counter2 = counter_factory()

print("Counter1:", counter1['increment'](5), counter1['get']())  
print("Counter2:", counter2['increment'](), counter2['get']())   
counter1['reset']()
print("After reset:", counter1['get'](), counter2['get']())      

预期结果为：

```bash
Counter1: 15 15
Counter2: 1 1
After reset: 0 1
```

与实际相符。

这里`counter1`调用`increment`与`get`函数得到15与15，随后`counter2`作为另外的frame调用`counter_factory`函数，并最终输出1与1，此时对`counter1`调用`reset`，但不影响`counter2`，最后输出0和1。

## 一、 函数基础练习-补充代码完善流程

### 1. 二分查找 代码填空

设计一个函数，实现二分查找 输入是一个升序的整数列表 和 一个数，返回这个数列表中的下标，没有查到打印 NA 

In [16]:
# 返回 x 在 arr 中的索引，如果不存在返回 -1
def binarySearch (arr, l, r, x): 
  
    # 基本判断
    if r >= l: 
  
        mid = l + (r - l) // 2  # 找到中点下标
  
        # 元素整好的中间位置
        if arr[mid] == x: 
            return mid 
          
        # 元素小于中间位置的元素，只需要再比较左边的元素
        elif arr[mid] > x: 
            return binarySearch(arr, l, mid-1, x)  # 递归向左半边             
  
        # 元素大于中间位置的元素，只需要再比较右边的元素
        else: 
            return binarySearch(arr, mid + 1, r, x)  # 递归向右半边 
  
    else: 
        # 不存在
        print('NA')
        return -1

# 测试数组
arr = [ 2, 3, 4, 10, 40 ] 
x = 10
  
# 函数调用
result = binarySearch(arr, 0, len(arr)-1, x) 
assert result == 3

### 2. 实现数字列表的归并排序 补充代码块

例如：
输入：[12,3,4,6,7,1,8,5]
输出：[1,3,4,5,6,7,8,12]

In [17]:
def mergesort(seq):
    """归并排序"""
    if len(seq) <= 1:
        return seq
    mid = len(seq) // 2  # 将列表分成更小的两个列表
    # 分别对左右两个列表进行处理，分别返回两个排序好的列表
    left = mergesort(seq[:mid])
    right = mergesort(seq[mid:])
    # 对排序好的两个列表合并，产生一个新的排序好的列表
    return merge(left, right)

def merge(left, right):
    """合并两个已排序好的列表，产生一个新的已排序好的列表"""
    result = []  # 新的已排序好的列表
    i = 0  # 下标
    j = 0
    
    # 对两个列表中的元素 两两对比。
    # 将最小的元素，放到result中，并对当前列表下标加1
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1 

    result += left[i:]  # <-list的加法即合并
    result += right[j:] 
    return result

seq = [12,3,4,6,7,1,8,5]
result = mergesort(seq)
assert result ==  [1, 3, 4, 5, 6, 7, 8, 12]

### 3.生成器实现两个序列的交错输出 补充实现函数体
    编写一个生成器函数，接受两个序列作为参数，并生成一个新序列，该序列交错包含原来两个序列的元素。例如，给定序列`[1, 3, 5]`和`[2, 4, 6]`，生成的序列应该是`[1, 2, 3, 4, 5, 6]`。（最好不要使用`zip`函数）

In [20]:
def interleave(seq1, seq2):
    """交叉合并两个列表"""
    result = []  
    length = min(len(seq1), len(seq2))

    for i in range(length):
        result.append(seq1[i])
        result.append(seq2[i])

    return result

assert list(interleave([1, 2, 3], [4, 5, 6,7 ])) == [1, 4, 2, 5, 3, 6]

### 4.使用装饰器计算函数运行的时间并输出，格式如下；

    如调用func(1,2,3)时，输出func(1,2,3):[0.06199502944946289]s;

    如调用func(1,[1,2],{1:[1,2,3]})时，输出func(1,[1, 2],{1: [1, 2, 3]}):[0.06698012351989746]s

提示：参数的字符化只考虑简单数据类型和包含简单数据类型的容器，并且只考虑参数*args，可以用map(str,args)转换为字符串。有基础的可以考虑试下**kwargs。

打印函数的名字可以使用func.__name__，获取当前时间可以使用time包

In [ ]:
import time
def log_time(func):
    def inner(*args):
        start_time = time.time()
        result = func(*args)
        end_time = time.time()
        execution_time = end_time - start_time

        # Convert arguments to strings, handling various types
        arg_strings = []

        for arg in args:
            if isinstance(arg, (list, tuple, dict, set)):
                arg_strings.append(repr(arg)) 
            else:
                arg_strings.append(str(arg))

        args_str = ", ".join(arg_strings) # whether ',' or ', ', that is a question...

        print(f"{func.__name__}({args_str}):[{execution_time}]s")
        return result 
    
    return inner

def log_time_mixed(func):
    def inner(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time

        # Convert *args to strings
        arg_strings = []

        for arg in args:
            if isinstance(arg, (list, tuple, dict, set)):
                arg_strings.append(repr(arg)) 
            else:
                arg_strings.append(str(arg))

        args_str = ", ".join(arg_strings)

        # Convert **kwargs to strings
        kwargs_strings = []
        # kwargs.items() can turn the keyword arguments into a sequence of tuples
        for key, value in kwargs.items(): 
            if isinstance(value, (list, tuple, dict, set)):
                kwargs_strings.append(f"{key}={repr(value)}")
            else:
                kwargs_strings.append(f"{key}={str(value)}")

        kwargs_str = ", ".join(kwargs_strings)

        # Combine args_str and kwargs_str
        if args_str and kwargs_str:
            full_args_str = f"{args_str}, {kwargs_str}"
        elif kwargs_str:
             full_args_str = kwargs_str
        else:
            full_args_str = args_str

        print(f"{func.__name__}({full_args_str}):[{execution_time}]s")
        return result 
    return inner

@log_time
def time_cosume(*args):
    return sum(range(1,1000000))

@log_time_mixed
def time_cosume(*args, **kwargs):
    return sum(range(1,1000000))

time_cosume(1,2,3,4,[1,2,3,4])
time_cosume(1,2,3,4,[1,2,3,4],{1:[1,2]})

time_cosume(1, 2, 3, 4, [1, 2, 3, 4]):[0.012638330459594727]s
time_cosume(1, 2, 3, 4, [1, 2, 3, 4], {1: [1, 2]}):[0.012788772583007812]s


499999500000